# 일출, 일몰 데이터 크롤링
- 일조량과 일사량 데이터의 결측치를 처리하기 위해서는, 해가 뜨는 일출시간과 해가 지는 일몰시간을 알아야한다.
- 일출시간과 일몰시간을 알지 못하면, 일조량과 일사량 데이터에서 해당 데이터가 일조량과 일사량이 없어서 결측치가 발생한 것인지, 관측을 하지 못해서 결측치가 발생한 것인지를 알 수 없게 된다.
- 그러므로, 일출시간과 일몰시간을 참조하여 일조량과 일사량의 데이터의 결측치를 처리해야 한다.
- 일출시간과 일몰시간 데이터는 한국천문연구원의 출몰시각 정보 API를 크롤링하여 가져왔다.
- 해당문서는 한국천문연구원의 출몰시각 정보 API를 크롤링하는 절차에 대해서 다룬다.

### 패키지로딩

In [1]:
### Crawling
import requests
from bs4 import BeautifulSoup

### Data Handling
import pandas as pd

### 크롤링에 필요한 날짜 데이터 로딩
- 우리가 분석하려는 데이터의 시간범위만큼 크롤링을 진행해야 될 것이다.
- 그러므로, 분석하려는 데이터의 일시를 가져와서 리스트형태로 저장한다.

In [2]:
datelist = pd.read_csv("C:/dust/preprocessing/2022_datelist.csv")
datelist = datelist["일시"].astype(str).unique().tolist()

### API 요청 함수 생성
- 서울의 일출, 일몰시간 데이터를 끌고오기 위해 API 서버에 요청한다.
- GET방식으로 크롤링이 가능하다.

In [3]:
import requests
def weather(date):
    url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'
    params ={
        'serviceKey' :"gcVGIzO9n58fC27nANUcTmbfzNXTt93wnEIn/0rLgOiW43nsH213u8dz2JOyvgS1iPxeAi3L69eseRiNQOPMJA==",
        'location' : '서울', 
        'locdate' : {date} 
        }
    res = requests.get(url, params=params)
    return res.text

### 크롤링 데이터 정제 함수 생성

In [4]:
def get_weather(date):
    soup = BeautifulSoup(weather(date))
    sunrise = int(soup.find('sunrise').text)
    sunset = int(soup.find('sunset').text)
    return sunrise, sunset

### 데이터 크롤링 및 저장

In [5]:
all_sunrise = []
all_sunset = []
for i in datelist :
    sunrise, sunset = get_weather(i)
    all_sunrise.append(sunrise)
    all_sunset.append(sunset)
    
pd.DataFrame({"date":datelist, "sunrise":all_sunrise, "sunset":all_sunset}).to_csv("sunrise_sunset_test.csv", index=False)

c:\Users\user\miniconda3\envs\env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
